# Keras, GridSearchCV

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

from keras.models import Sequential
from keras.layers import Dense , Dropout
from keras.optimizers import RMSprop

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.


## Data load & transform

In [2]:
train = (pd.read_csv('./input/train.csv')).astype('float32')
print "train shape:", train.shape

train shape: (42000, 785)


In [3]:
X, Y = train.drop('label', axis=1).values, train['label'].values

print "X shape:", X.shape
print "Y shape:", Y.shape

X shape: (42000, 784)
Y shape: (42000,)


In [4]:
# Reduce datapoints
X = X[:5000,:]
Y = Y[:5000]

print "X shape:", X.shape
print "Y shape:", Y.shape

X shape: (5000, 784)
Y shape: (5000,)


In [5]:
# Normalization
X = X / 255

In [6]:
from keras.utils.np_utils import to_categorical
Y = to_categorical(Y, 10)

print "Y shape:", Y.shape

Y shape: (5000, 10)


## NN

In [7]:
# fix random seed for reproducibility
seed = 9
np.random.seed(seed)

In [8]:
def create_model(lr, rho, epsilon, decay):
    
    # create model
    model = Sequential()
    model.add(Dense(64, activation='relu', input_dim=(28 * 28)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.15))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.15))
    model.add(Dense(10, activation='softmax'))
    
    
    
    # Compile model
    model.compile(
        optimizer=RMSprop(lr=lr, rho=rho, epsilon=epsilon, decay=decay),
        loss = 'categorical_crossentropy',
        metrics = ['accuracy']
    )    

    return model

In [9]:
# create model
keras_classifier = KerasClassifier(
    build_fn = create_model, 
    verbose = 0
)

In [10]:
grid = GridSearchCV(
    estimator = keras_classifier, 
    param_grid = dict(
#         epochs = [5, 10, 15],
#         batch_size = [5, 10, 20],
#         optimizer = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
        epochs = [15],
        batch_size = [10],
#         optimizer = ['SGD', 'Adagrad', 'Adam', 'Nadam']
#         optimizer = ['RMSprop']
        lr = [0.00001, 0.001, 0.1],
        rho = [0.5, 0.9, 1.3],
        epsilon = [1e-07, 1e-08, 1e-09],
        decay = [0.0, 0.2, 0.5]
    ),
    cv = 10,
    n_jobs=1, 
    verbose=2
)

In [11]:
grid_result = grid.fit(X, Y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 81 candidates, totalling 810 fits
[CV] decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5 
[CV]  decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5, total=  14.9s
[CV] decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.3s remaining:    0.0s


[CV]  decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5, total=  14.4s
[CV] decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5 
[CV]  decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5, total=  13.8s
[CV] decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5 
[CV]  decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5, total=  13.0s
[CV] decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5 
[CV]  decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5, total=  13.5s
[CV] decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5 
[CV]  decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5, total=  13.0s
[CV] decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5 
[CV]  decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5, total=  13.4s
[CV] decay=0.0, epsilon=1e-07, batch_size=10, epochs=15, lr=1e-05, rho=0.5

[Parallel(n_jobs=1)]: Done 810 out of 810 | elapsed: 527.1min finished


Best: 0.939600 using {'decay': 0.0, 'epsilon': 1e-09, 'batch_size': 10, 'epochs': 15, 'lr': 0.001, 'rho': 0.9}
0.741800 (0.031054) with: {'decay': 0.0, 'epsilon': 1e-07, 'batch_size': 10, 'epochs': 15, 'lr': 1e-05, 'rho': 0.5}
0.783200 (0.023953) with: {'decay': 0.0, 'epsilon': 1e-07, 'batch_size': 10, 'epochs': 15, 'lr': 1e-05, 'rho': 0.9}
0.100800 (0.015367) with: {'decay': 0.0, 'epsilon': 1e-07, 'batch_size': 10, 'epochs': 15, 'lr': 1e-05, 'rho': 1.3}
0.934800 (0.007756) with: {'decay': 0.0, 'epsilon': 1e-07, 'batch_size': 10, 'epochs': 15, 'lr': 0.001, 'rho': 0.5}
0.936000 (0.008809) with: {'decay': 0.0, 'epsilon': 1e-07, 'batch_size': 10, 'epochs': 15, 'lr': 0.001, 'rho': 0.9}
0.098800 (0.011179) with: {'decay': 0.0, 'epsilon': 1e-07, 'batch_size': 10, 'epochs': 15, 'lr': 0.001, 'rho': 1.3}
0.095000 (0.012969) with: {'decay': 0.0, 'epsilon': 1e-07, 'batch_size': 10, 'epochs': 15, 'lr': 0.1, 'rho': 0.5}
0.093000 (0.007759) with: {'decay': 0.0, 'epsilon': 1e-07, 'batch_size': 10, 'e